In [ ]:
from lib.functions0 import *
from lib.utility import *
import numpy as np
from scipy.linalg import expm
from qiskit.utils.mitigation.fitters import CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import  complete_meas_cal
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
import matplotlib.pyplot as plt
import array_to_latex
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community',
                            group='ibmquantumawards', 
                            project='open-science-22')

backend_sim_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_real_jakarta = provider.get_backend('ibmq_jakarta')
backend_sim = Aer.get_backend('qasm_simulator')

In [ ]:
n_steps=100
time=np.pi
#initial_state={"110": 1}
shots = 32000
backend = backend_sim_jakarta

X = np.array([[0,1],[1,0]])  #defining the pauli matrices
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
Id = np.eye(2)

# defining the hamiltonian divided in: 
#       - H1: first two qubits interactions.
#       - H2: second two qubits interactions.

H1 = np.kron(X, np.kron(X,Id)) + np.kron(Y, np.kron(Y,Id)) + np.kron(Z, np.kron(Z,Id)) 
H2 = np.kron(Id, np.kron(X,X)) + np.kron(Id, np.kron(Y,Y)) + np.kron(Id, np.kron(Z,Z)) 

# building numerically the trotter step matrix, and the whole operator (trotter step)^n_steps.

trotter_step_matrix_= expm(-time/n_steps*H1*1j).dot(expm(-time/n_steps*H2*1j))
trotterized = np.linalg.matrix_power(trotter_step_matrix_, n_steps)
Matrix(trotterized).n(3, chop=True)

In [ ]:
A0 = trotterized[3, 6]
A1 = trotterized[5, 6]
A2 = trotterized[6, 6]

r1=float(atan2(im(A0),re(A0))+atan2(im(A2),re(A2)))/2
r2=0
f1=float(atan2(im(A2),re(A2))-atan2(im(A1),re(A1))-np.pi)/2
f2=float((atan2(im(A2),re(A2))-atan2(im(A0),re(A0)))/2-f1)
a1=float(acos(abs(A2)))
a2=float(acos(abs(A1)/sin(a1)))

M1_qc = fixed_magnetization_two_qubit_gate(r1,f1,a1)
M2_qc = fixed_magnetization_two_qubit_gate(r2,f2,a2)

## building the operator U gate

In [ ]:

qr_U = QuantumRegister(3 ,name="q")
qc_U = QuantumCircuit(qr_U, name="U")

qc_U.append(M1_qc, [qr_U[0],qr_U[1]])
qc_U.append(M2_qc, [qr_U[1],qr_U[2]])

qc_U=transpile(qc_U, basis_gates=["cx","rz","x","sx"])
qc_U.draw(output="mpl")

In [ ]:
U_ideal=matrix_from_circuit(qc_U, type="numpy")
Matrix(U_ideal)

## building the evolution circuit

In [ ]:
qr_evo = QuantumRegister(3, name="q")
qc_evo = QuantumCircuit(qr_evo, name="U")

qc_evo.x([qr_evo[1],qr_evo[2]])
qc_evo.append(qc_U, qr_evo)

qcs_tomo = state_tomography_circuits(qc_evo, qr_evo)
qcs_tomo[3].draw(output="mpl")

## building the calibration matrices

In [ ]:
qr_cal = QuantumRegister(3)
cal_circ, state_labels = complete_meas_cal(qubit_list=[0,1,2], qr=qr_cal, circlabel='mcal')

## building the itself calibration circuit: *Circuit-Itself-Calibration (CIC)*

In [ ]:
Number_of_tomography_basis = 3**3

circuits_calibration_itself = list(range(Number_of_tomography_basis))

for tomography_base in range(Number_of_tomography_basis):

    circuits_calibration_itself[tomography_base] = []

    for i in state_labels:

        cr_cal_itself = ClassicalRegister(3)
        qr_cal_itself = QuantumRegister(3)
        qc_cal_itself = QuantumCircuit(qr_cal_itself, cr_cal_itself, name=f"mcalcal_{i}") # changing the name to not have API contrast

        for k in range(3):
            if i[::-1][k] == "1":
                qc_cal_itself.x(qr_cal_itself[k])

        qc_cal_itself.append(qc_U, qr_cal_itself)
        qc_cal_itself.measure(qr_cal_itself, cr_cal_itself)

        circuits_calibration_itself[tomography_base].append(qc_cal_itself)

    


## building the identity calibration circit: *Noisy-Identity-Calibration (NIC)*

In [ ]:
calib_circuits_identity = []

for i in state_labels:

    cr_cal_id = ClassicalRegister(3)
    qr_cal_id = QuantumRegister(3)
    qc_cal_id = QuantumCircuit(qr_cal_id, cr_cal_id, name=f"mcalcal_{i}")

    qc_cal_id.append(qc_U, qr_cal_id)

    for k in range(3):
        if i[::-1][k] == "1":
            qc_cal_id.x(qr_cal_id[k])
        
    qc_cal_id.measure(qr_cal_id, cr_cal_id)

    calib_circuits_identity.append(qc_cal_id)

## execution of the circuits

In [ ]:
job_tomo=execute(qcs_tomo, backend, shots=shots, initial_layout=[1,3,5])

job_cal_our_identity=execute(calib_circuits_identity, backend = backend, shots=shots, initial_layout=[1,3,5])

jobs_cal_our_itself = []
for tomography_base_calibration_circuits in circuits_calibration_itself:
    job_tomography_itself=execute(tomography_base_calibration_circuits, backend = backend, shots=shots, initial_layout=[1,3,5])
    jobs_cal_our_itself.append(job_tomography_itself)

job_cal_qiskit=execute(cal_circ, backend, shots=shots, initial_layout=[1,3,5])

## building the matrices of the probabilies for the qiskit, identity and circuit-itself calibrations

In [ ]:
meas_fitter_identity = CompleteMeasFitter(job_cal_our_identity.result(), state_labels=state_labels)

meas_fitter_itself_list = []
for job_cal_our_itself in jobs_cal_our_itself:
    meas_fitter_itself_list.append(CompleteMeasFitter(job_cal_our_itself.result(), state_labels=state_labels))

meas_fitter_qiskit = CompleteMeasFitter(job_cal_qiskit.result(), state_labels=state_labels)

## building the calibration matrix for each circuit of the tomography

In [ ]:
U_tilde_identity=meas_fitter_identity.cal_matrix

U_tilde_itself_list = []
for meas_fitter_itself in meas_fitter_itself_list:
    U_tilde_itself_list.append(meas_fitter_itself.cal_matrix)

U_tilde_qiskit=meas_fitter_qiskit.cal_matrix

In [ ]:
def matrix_from_cirquit(qc, phase=0, type="sympy"):

    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend, shots=32000)
    result = job.result()
    A=result.get_unitary(qc, decimals=10)*np.exp(1j*phase)
    if type=="sympy":
        return Matrix(A)
    else:
        return A

## building the matrix of the tomography basis 


In [ ]:
qr_basi = QuantumRegister(3)
qc_basi = QuantumCircuit(qr_basi)

qcs_basis = state_tomography_circuits(qc_basi, qr_basi)

In [ ]:
C_matrices_itself = []
C_matrices_identity = []

C_identity = U_tilde_identity

for tomography_base in range(Number_of_tomography_basis):

    print(tomography_base)
    qcs_basis[tomography_base].remove_final_measurements()
    base_matrix = matrix_from_circuit(qcs_basis[tomography_base], type="numpy")
    base_matrix_inverse = np.linalg.inv(base_matrix)

    # inverting the ideal matrix of the circuit

    U_ideal_inverse = np.linalg.inv(U_ideal)
    U_ideal_inverse_abs = np.abs(U_ideal_inverse)**2

    U_ideal_abs = np.abs(U_ideal)**2
    U_ideal_abs_inverse = np.linalg.inv(U_ideal_abs)

    base_matrix_inverse_abs = np.abs(base_matrix_inverse)

    U_ideal_H_abs_inverse = np.dot(U_ideal_inverse_abs, base_matrix_inverse_abs)

    #U_ideal_H_abs = np.abs(U_ideal_H)**2
    #U_ideal_H_inv = np.linalg.inv(U_ideal_H_abs)

    

    C_itself = np.dot(U_tilde_itself_list[tomography_base], U_ideal_H_abs_inverse)

    #C_aus_itself = np.linalg.multi_dot([base_matrix, C_itself,  base_matrix_H])
    #C_aus_itself = np.linalg.multi_dot([U_tilde_qiskit, base_matrix, np.linalg.inv(U_tilde_qiskit), C_itself,  base_matrix_inverse])
    #C_aus_itself = np.linalg.multi_dot([base_matrix, U_tilde_qiskit, np.linalg.inv(U_tilde_qiskit), C_itself])
    C_aus_identity = np.matmul(base_matrix, np.matmul(C_identity, base_matrix_inverse))

    C_matrices_identity.append(np.asarray(C_aus_identity))
    C_matrices_itself.append(np.asarray(C_itself))



In [ ]:
from copy import deepcopy

meas_fitters_identity = []
meas_fitters_itself = []

for C_new in C_matrices_identity:
    meas_fitter_our_aus = deepcopy(meas_fitter_identity)
    meas_fitter_our_aus._tens_fitt.cal_matrices[0]=C_new

    meas_fitters_identity.append(meas_fitter_our_aus)

for i, C_new in enumerate(C_matrices_itself):
    meas_fitter_our_aus = deepcopy(meas_fitter_itself_list[0])
    meas_fitter_our_aus._tens_fitt.cal_matrices[0]=C_new

    meas_fitters_itself.append(meas_fitter_our_aus)

In [ ]:
target_state = (One^One^Zero).to_matrix()
#target_state = (Zero^One^One).to_matrix()


fids=np.zeros(4)

raw_res=deepcopy(job_tomo.result())
qiskit_res=deepcopy(raw_res)
identity_res=deepcopy(raw_res)
itself_res=deepcopy(raw_res)

In [ ]:
for i in range(27):

    old_counts=raw_res.get_counts(i)
    new_counts_qiskit = meas_fitter_qiskit.filter.apply(old_counts, method="least_squares")
    qiskit_res.results[i].data.counts = new_counts_qiskit

for i in range(27):

    old_counts=raw_res.get_counts(i)
    
    new_counts_id = meas_fitters_identity[i].filter.apply(old_counts, method="least_squares")
    identity_res.results[i].data.counts = new_counts_id

    new_counts_it = meas_fitters_itself[i].filter.apply(old_counts, method="least_squares")
    itself_res.results[i].data.counts = new_counts_it

In [ ]:
identity_res.get_counts(0)

In [ ]:
itself_res.get_counts(-1)

In [ ]:
fids[0] = fidelity_count(raw_res, qcs_tomo, target_state)
fids[1] = fidelity_count(qiskit_res, qcs_tomo, target_state)
fids[2] = fidelity_count(identity_res, qcs_tomo, target_state)
fids[3] = fidelity_count(itself_res, qcs_tomo, target_state)

fids